In [9]:
from hybrid import ListCombinationHybrid
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender
from Data_Handler.DataReader import DataReader
from evaluator import evaluate
from tqdm import tqdm
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

dataReader = DataReader()

target = dataReader.load_target()
URM = dataReader.load_augmented_binary_urm()
URM_aug, ICM = dataReader.pad_with_zeros_ICMandURM(URM)
URM_train_aug, URM_validation = split_train_in_two_percentage_global_sample( URM_aug, train_percentage=1.0)
URM_train_pow = dataReader.stackMatrixes(URM_train_aug)

ICM_stacked_with_weighted_impressions = dataReader.load_ICM_stacked_with_weighted_impressions()
URM_train_aug, ICM = dataReader.pad_with_zeros_given_ICMandURM(ICM_stacked_with_weighted_impressions, URM_train_aug) 
URM_train_super_pow = dataReader.load_super_powerful_URM(URM_train_aug, ICM_stacked_with_weighted_impressions, 0.8)

--->  23091
==> 24507


In [10]:
SSLIM_pow = SLIMElasticNetRecommender(URM_train_pow)
SSLIM_pow.fit()

SLIMElasticNetRecommender: Processed 2282 ( 8.2%) in 5.00 min. Items per second: 7.60
SLIMElasticNetRecommender: Processed 4467 (16.0%) in 10.00 min. Items per second: 7.44
SLIMElasticNetRecommender: Processed 6791 (24.3%) in 15.01 min. Items per second: 7.54
SLIMElasticNetRecommender: Processed 9203 (32.9%) in 20.01 min. Items per second: 7.67
SLIMElasticNetRecommender: Processed 11670 (41.7%) in 25.01 min. Items per second: 7.78
SLIMElasticNetRecommender: Processed 14042 (50.2%) in 30.01 min. Items per second: 7.80
SLIMElasticNetRecommender: Processed 16445 (58.8%) in 35.01 min. Items per second: 7.83
SLIMElasticNetRecommender: Processed 18841 (67.4%) in 40.01 min. Items per second: 7.85
SLIMElasticNetRecommender: Processed 21166 (75.7%) in 45.01 min. Items per second: 7.84
SLIMElasticNetRecommender: Processed 23602 (84.4%) in 50.01 min. Items per second: 7.86
SLIMElasticNetRecommender: Processed 27968 (100.0%) in 54.32 min. Items per second: 8.58


In [11]:
SSLIM_super_pow = SLIMElasticNetRecommender(URM_train_super_pow)
SSLIM_super_pow.fit(l1_ratio=0.01,alpha=0.001, topK=870)

SLIMElasticNetRecommender: Processed 1746 ( 6.4%) in 5.00 min. Items per second: 5.82
SLIMElasticNetRecommender: Processed 3571 (13.1%) in 10.00 min. Items per second: 5.95
SLIMElasticNetRecommender: Processed 5488 (20.1%) in 15.00 min. Items per second: 6.10
SLIMElasticNetRecommender: Processed 7319 (26.8%) in 20.01 min. Items per second: 6.10
SLIMElasticNetRecommender: Processed 9170 (33.6%) in 25.01 min. Items per second: 6.11
SLIMElasticNetRecommender: Processed 10972 (40.2%) in 30.01 min. Items per second: 6.09
SLIMElasticNetRecommender: Processed 12603 (46.2%) in 35.01 min. Items per second: 6.00
SLIMElasticNetRecommender: Processed 13511 (49.5%) in 1.04 hour. Items per second: 3.62
SLIMElasticNetRecommender: Processed 14257 (52.2%) in 1.12 hour. Items per second: 3.53
SLIMElasticNetRecommender: Processed 15128 (55.4%) in 1.20 hour. Items per second: 3.49
SLIMElasticNetRecommender: Processed 15951 (58.4%) in 1.29 hour. Items per second: 3.44
SLIMElasticNetRecommender: Processed 1

In [13]:
recommender = ListCombinationHybrid(URM_train_aug,SSLIM_pow,SSLIM_super_pow)
recommender.fit()

f = open("submission.csv", "w+")
f.write("user_id,item_list\n")
recommended_items_for_each_user = {}
for user_id in tqdm(target):
    recommended_items = recommender.recommend(
        user_id, cutoff=10, remove_seen_flag=True)
    recommended_items_for_each_user[int(user_id)] = recommended_items
    well_formatted = " ".join([str(x) for x in recommended_items])
    f.write(f"{user_id}, {well_formatted}\n")

# Evaluate recommended items
map = evaluate(recommended_items_for_each_user, URM_validation, target)
print('MAP score: {}'.format(map))

ListCombinationHybrid: URM Detected 2785 (10.2%) items with no interactions.


  0%|          | 0/41116 [00:00<?, ?it/s]


IndexError: index 0 is out of bounds for axis 0 with size 0